In [79]:
#base_packs
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import statsmodels.api as sm
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from statsmodels.stats.diagnostic import het_white
import ruptures as rpt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import statsmodels.api as sm
import sklearn.preprocessing as skp
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
from sklearn.cluster import KMeans
import gc
import warnings
from scipy.optimize import OptimizeWarning
import time
import numpy as np
import random
from sklearn.preprocessing import RobustScaler, QuantileTransformer,MinMaxScaler
from sklearn.metrics import calinski_harabasz_score

In [80]:
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
dt['Tempo de depressão de vácuo(min)']= pd.to_numeric(dt['Tempo de depressão de vácuo(min)'] , errors = 'coerce')

dt['Teor de C após a descarburação(%)'].replace(',' , '.', inplace = True)
dt['Teor de C após a descarburação(%)']= pd.to_numeric(dt['Teor de C após a descarburação(%)'] , errors = 'coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1 , inplace = True)
dt.replace('Ruim', 0 , inplace = True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()  
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})

In [82]:
teste = 'C:/Users/S409478/Documents/vm_report_vo1727442932637.xls'
teste = pd.read_excel(teste)
teste = pd.DataFrame(teste)
# Extract the first row and set as new header
new_header = teste.iloc[0]
teste = teste[1:]  # Drop the first row
teste.columns = new_header  # Set new header

# Reset index
teste.reset_index(drop=True, inplace=True)

# Function to format numbers
def insert_decimal(x):
    x_str = str(x)
    
    # Handle cases where there is already a decimal point
    if '.' in x_str:
        integer_part, decimal_part = x_str.split('.')
        # If the integer part is shorter than 2 digits, just return as is
        if len(integer_part) <= 3:
            return x_str
        # Otherwise, insert decimal point after the second-to-last digit
        return integer_part[:-3] + '.' + integer_part[-3:] + (f".{decimal_part}" if decimal_part else "")
    
    # Handle integer values
    if len(x_str) > 3:
        return x_str[:-3] + '.' + x_str[-3:]
    return x_str

# Apply the function to each element in the DataFrame column
teste['Tempo abaixo de 10 mbar(min)'] = teste['Tempo abaixo de 10 mbar(min)'].apply(insert_decimal)
#teste['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
teste.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação','AÇO'], inplace=True)

In [83]:
de = 'C:/Users/S409478\Documents/vm_report_vo1728644376336.xls'
de = pd.read_excel(de)
de = pd.DataFrame(de)
# Extract the first row and set as new header
new_header = de.iloc[0]
de = de[1:]  # Drop the first row
de.columns = new_header  # Set new header

# Reset index
de.reset_index(drop=True, inplace=True)

# Function to format numbers
def insert_decimal(x):
    x_str = str(x)
    
    # Handle cases where there is already a decimal point
    if '.' in x_str:
        integer_part, decimal_part = x_str.split('.')
        # If the integer part is shorter than 2 digits, just return as is
        if len(integer_part) <= 3:
            return x_str
        # Otherwise, insert decimal point after the second-to-last digit
        return integer_part[:-3] + '.' + integer_part[-3:] + (f".{decimal_part}" if decimal_part else "")
    
    # Handle integer values
    if len(x_str) > 3:
        return x_str[:-3] + '.' + x_str[-3:]
    return x_str

# Apply the function to each element in the DataFrame column
de['Tempo abaixo de 10 mbar(min)'] = de['Tempo abaixo de 10 mbar(min)'].apply(insert_decimal)
#teste['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
de.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação','AÇO'], inplace=True)
#teste = pd.to_numeric(teste, errors='coerce')
de = teste.apply(pd.to_numeric, errors='coerce')
de = teste.dropna(how = 'any')
de = teste.reset_index(drop=True)

In [84]:
#teste = pd.to_numeric(teste, errors='coerce')
teste = teste.apply(pd.to_numeric, errors='coerce')
teste = teste.dropna(how = 'any')
teste = teste.reset_index(drop=True)

In [85]:
##normalization
scaler = skp.RobustScaler()
dt_k = scaler.fit_transform(dt)
kmeans = KMeans(n_clusters=5 , random_state=0)

# Fit the model to the data
kmeans.fit(dt_k)

# Get cluster centers and labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_
dt['cluster'] = labels 
tag =pd.DataFrame(dt['cluster'])
tag = pd.get_dummies(dt['cluster'], prefix='cluster')
tag.replace({False: 0, True: 1})
ch_index = calinski_harabasz_score(dt_k, labels)
ch_index

1902.5803728531569

In [86]:
##normalization
scaler = skp.RobustScaler()
dt_k = scaler.fit_transform(teste)
kmeans = KMeans(n_clusters=5 , random_state=0)

# Fit the model to the data
kmeans.fit(dt_k)

# Get cluster centers and labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_
teste['cluster'] = labels 
tag =pd.DataFrame(teste['cluster'])
tag = pd.get_dummies(teste['cluster'], prefix='cluster')
tag.replace({False: 0, True: 1})
ch_index = calinski_harabasz_score(dt_k, labels)
ch_index

559.702286255723

In [87]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
dt['opt'] = 0
dt.loc[
    (dt['Teor de C após a descarburação(%)'] < 0.009) &
(dt['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(dt['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1


In [88]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
teste['opt'] = 0
teste.loc[
    (teste['Teor de C após a descarburação(%)'] < 0.009) &
(teste['Temperatura do aço após a descarburação(°C)'] >= 1680) &
(teste['Temperatura do aço após a descarburação(°C)'] <= 1700)
    ,
    'opt'
] = 1

In [89]:
teste['opt'].value_counts()


opt
0    34
1    16
Name: count, dtype: int64

In [9]:
dt.columns

Index(['Unnamed: 0', 'CORRIDA', 'Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)',
       'Teor de Mo antes do VOD(%)', 'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela',
       'Vazão média de argônio durante o sopro de O2(Nm3/h)',
       'Pressão média de vácuo durante o sopro de O2(mbar)',
       'Temperatura máxima dos gases durante o sopro de O2(°C)',
       'Pressão final de vácuo do sopro de O2(mbar)',
       'Tempo de depressão de vácuo(min)', 'Tempo total de descarburação(min)',
       'Tempo abaixo de 10 mbar(min)',
       'Vazão média de argônio durante a descarburação(Nm3/h)',
       'Pressão final de vácuo na descarburação(mbar)',
       '%CO no final da descarburação', 'Volume de O2 soprado(m3)',
       'Teor de C após a descarburação(%)',
       'Temperat

In [150]:
# Primeira volta Hubber, teste:
##OK
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import warnings

warnings.filterwarnings('ignore')

# Custom Huber loss function for logistic regression
def huber_loss(params, X, y, delta=1.0):
    """Calculate Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    """Gradient of Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Required columns for the model
required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)','Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)','Temperatura máxima dos gases durante o sopro de O2(°C)','Tempo total de descarburação(min)','Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)'
]

# Define control DataFrame
columns_control = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)','Temperatura máxima dos gases durante o sopro de O2(°C)','Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)',
]

# Example control values
stp = [[ 4.9  , 465.6  ,  16.717,   5.97 , 184.   , 278.64 , 248.17 ,
          5.233,   2.   ]]
contr = pd.DataFrame(stp, columns=columns_control)

# Prepare input DataFrame
ent = teste[['Teor de C antes do VOD(%)',
       'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
       'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)','Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
       'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
       'Vida de plug da panela', 'Vida da panela','Tempo total de descarburação(min)']]

#ent_ = ent.sample(n=1)

# Combine control and input data
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent_, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T

# Prepare test data for predictions
tt2_test = tt2[columns_control]

# Prepare Data for training
x_ = dt[required_columns]
y_ = dt['opt']

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.1, random_state=42)
#teste_size = 0.1

# Optimize using Huber loss
initial_params = np.zeros(tt2.shape[1])
result = minimize(fun=huber_loss, x0=initial_params, args=(x_resampled.values, y_resampled.values),
                  jac=huber_loss_grad, method='BFGS')

optimal_params = result.x

# Predicting on tt2
tt2_features = tt2.values
y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

# Display predictions
tt2['Predicted_Probability'] = y_pred_tt2_prob
tt2['Predicted_Class'] = y_pred_tt2_binary

print("Predictions on tt2:")
print(tt2[['Predicted_Probability', 'Predicted_Class']], ent_.index)

Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.999979                1 Index([11], dtype='int64')


In [145]:
teste.iloc[ent_.index][['opt','Teor de C após a descarburação(%)',
       'Temperatura do aço após a descarburação(°C)']]

,opt,Teor de C após a descarburação(%),Temperatura do aço após a descarburação(°C)
11,1,0.008,1692.0


In [143]:
teste.iloc[ent_.index][columns_control]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3),Temperatura máxima dos gases durante o sopro de O2(°C),Pressão final de vácuo do sopro de O2(mbar),Tempo de depressão de vácuo(min),Pressão final de vácuo na descarburação(mbar)
11,4.9,465.6,16.717,5.97,184.0,278.64,248.17,5.233,2.0


In [138]:
from imblearn.over_sampling import ADASYN

# Initialize ADASYN
adasyn = ADASYN(random_state=42)

# Apply ADASYN to the dataset
x_resampled, y_resampled = adasyn.fit_resample(x_, y_)

In [108]:
teste.iloc[ent_.index][columns_control].values

array([[  4.9   , 518.2   ,  18.167 ,   5.67  , -14.    ,  52.7167]])

In [164]:
# custom regularization:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time
import warnings
import gc

# Custom regularization function to discourage low coefficients
def custom_regularization(params, threshold=0.1, lambda_=0.01):
    """Custom regularization function to discourage low coefficients."""
    return lambda_ * np.sum(np.clip(np.abs(params) - threshold, 0, None) ** 2)

# Custom Huber loss function for logistic regression with regularization
def huber_loss(params, X, y, delta=1.0, lambda_=0.01, threshold=0.1):
    """Calculate Huber loss for logistic regression with custom regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    
    # Custom regularization term
    regularization_term = custom_regularization(params, threshold, lambda_)
    
    return (np.sum(loss) / n) + regularization_term

# Gradient of Huber loss with regularization
def huber_loss_grad(params, X, y, delta=1.0, lambda_=0.01, threshold=0.1):
    """Gradient of Huber loss for logistic regression with custom regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    
    # Gradient of custom regularization
    reg_grad = lambda_ * np.where(np.abs(params) < threshold, 0, params - threshold * np.sign(params))
    
    grad += reg_grad
    
    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)','Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)','Temperatura máxima dos gases durante o sopro de O2(°C)','Tempo total de descarburação(min)','Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)'
]


        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Initialize parameters for the optimization
        initial_params = np.zeros(x_train_3.shape[1])  # Ensure correct shape based on training data

        # Set robust loss parameters
        delta = 1.0  # The threshold at which the loss function changes from quadratic to linear
        lambda_ = 0.01  # Regularization strength
        threshold = 0.1  # Threshold for discouraging low coefficients

        # Optimize using Huber loss with custom regularization
        result = minimize(
            fun=huber_loss,
            x0=initial_params,
            args=(x_train.values, y_train.values, delta, lambda_, threshold),
            jac=huber_loss_grad,
            method='BFGS'
        )

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train_3, x_train_3)
        
        results = logit_model.fit(method='bfgs', cov_type='HC3')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_tt2_prob >= 0.99) and np.all(y_pred_tt2_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.945021                1
         Current function value: 0.240660
         Iterations: 35
         Function evaluations: 49
         Gradient evaluations: 40
Iteration: 2, Time elapsed: 0.27 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.945021                1
         Current function value: 0.271754
         Iterations: 35
         Function evaluations: 50
         Gradient evaluations: 41
An error occurred: single positional indexer is out-of-bounds
Iteration: 3, Time elapsed: 0.35 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.945021                1
         Current function value: 0.222721
         Iterations: 35
         Function evaluations: 49
         Gradient evaluations: 40
Iteration: 4, Time elapsed: 0.60 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Clas

In [165]:
print("Predictions on tt2:")
print(tt2[['Predicted_Probability', 'Predicted_Class']])
print(tt2[columns_control].values)

Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.960069                1
[[  6.82518915 693.31649647  23.           7.87       157.85035306
  278.64       248.17         5.233        2.        ]]


In [166]:
results_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    opt   No. Observations:                   51
Model:                          Logit   Df Residuals:                       30
Method:                           MLE   Df Model:                           20
Date:                Fri, 11 Oct 2024   Pseudo R-squ.:                  0.3464
Time:                        13:38:28   Log-Likelihood:                -18.921
converged:                      False   LL-Null:                       -28.950
Covariance Type:                  HC3   LLR p-value:                    0.4543
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Teor de C antes do VOD(%)                                  2.0626     11.659      0.177      0.860     -20.788      24.913
Teor de Cr antes do VOD(%)                                 2.2695      1.544      1.470      0.142      -0.757       5.296
Teor de Si antes do VOD(%)                                -0.9046     27.619     -0.033      0.974     -55.037      53.228
Teor de Mn antes do VOD(%)                                -2.8510      9.874     -0.289      0.773     -22.205      16.503
Teor de Ni antes do VOD(%)                               -11.4422      6.388     -1.791      0.073     -23.963       1.078
Teor de Mo antes do VOD(%)                                -3.7125      7.915     -0.469      0.639     -19.225      11.800
Temperatura do aço antes do VOD(°C)                        0.0118      0.021      0.557      0.577      -0.030       0.053
Peso de aço antes do VOD(kg)                               0.0008      0.002      0.460      0.646      -0.003       0.004
Borda livre da panela(mm)                                  0.0051      0.003      1.774      0.076      -0.001       0.011
Vida de plug da panela                                    -0.7167      0.360     -1.990      0.047      -1.423      -0.011
Vida da panela                                            -0.0537      0.173     -0.310      0.756      -0.393       0.286
Vazão média de argônio durante o sopro de O2(Nm3/h)       -0.9153      0.586     -1.561      0.118      -2.064       0.234
Pressão média de vácuo durante o sopro de O2(mbar)         0.0125      0.010      1.278      0.201      -0.007       0.032
Tempo abaixo de 10 mbar(min)                               0.7356      0.455      1.617      0.106      -0.156       1.627
Vazão média de argônio durante a descarburação(Nm3/h)      0.9108      0.813      1.120      0.263      -0.683       2.504
Volume de O2 soprado(m3)                                   0.0174      0.070      0.248      0.804      -0.120       0.155
Temperatura máxima dos gases durante o sopro de O2(°C)     0.0004      0.028      0.014      0.989      -0.055       0.056
Tempo total de descarburação(min)                         -0.7788      0.503     -1.548      0.122      -1.765       0.207
Pressão final de vácuo do sopro de O2(mbar)               -0.0103      0.008     -1.231      0.218      -0.027       0.006
Tempo de depressão de vácuo(min)                           0.1938      0.234      0.827      0.408      -0.266       0.653
Pressão final de vácuo na descarburação(mbar)             -0.3379      0.282     -1.196      0.232      -0.891       0.216
==========================================================================================================================
"""

In [170]:
logit_model = sm.Logit(y_, x_)
        
results = logit_model.fit(method='bfgs', cov_type='HC3',
penalty='l2', solver='liblinear')

# Extract coefficients and compute exponentiated coefficients
co = results.params
exp_coeff = np.exp(co) / (np.exp(co) + 1)
exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

# Extract results summary
results_summary = results.summary()
results_as_html = results_summary.tables[1].as_html()
df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_results['exp_coeff'] = exp_coeff_df

         Current function value: 0.483954
         Iterations: 35
         Function evaluations: 51
         Gradient evaluations: 42


In [171]:
df_results

,coef,std err,z,P>|z|,[0.025,0.975],exp_coeff
Teor de C antes do VOD(%),0.164500,1.331,0.124,0.902,-2.444,2.773,0.541035
Teor de Cr antes do VOD(%),0.224100,0.397,0.564,0.572,-0.554,1.002,0.555780
Teor de Si antes do VOD(%),0.291400,3.314,0.088,0.930,-6.204,6.787,0.572338
Teor de Mn antes do VOD(%),0.459400,2.399,0.191,0.848,-4.243,5.162,0.612868
Teor de Ni antes do VOD(%),-1.923000,0.846,-2.272,0.023,-3.582,-0.264,0.127528
Teor de Mo antes do VOD(%),0.142600,1.480,0.096,0.923,-2.759,3.044,0.535581
Temperatura do aço antes do VOD(°C),0.001400,0.004,0.372,0.710,-0.006,0.009,0.500355
Peso de aço antes do VOD(kg),0.000042,0.000,0.274,0.784,-0.000,0.000,NaN
Borda livre da panela(mm),0.001800,0.001,2.766,0.006,0.001,0.003,0.500459
Vida de plug da panela,0.021000,0.062,0.337,0.736,-0.101,0.143,0.505246


In [167]:
   df_results

,coef,std err,z,P>|z|,[0.025,0.975],exp_coeff
Teor de C antes do VOD(%),2.0626,11.659,0.177,0.860,-20.788,24.913,0.887220
Teor de Cr antes do VOD(%),2.2695,1.544,1.470,0.142,-0.757,5.296,0.906321
Teor de Si antes do VOD(%),-0.9046,27.619,-0.033,0.974,-55.037,53.228,0.288109
Teor de Mn antes do VOD(%),-2.8510,9.874,-0.289,0.773,-22.205,16.503,0.054628
Teor de Ni antes do VOD(%),-11.4422,6.388,-1.791,0.073,-23.963,1.078,0.000011
Teor de Mo antes do VOD(%),-3.7125,7.915,-0.469,0.639,-19.225,11.800,0.023835
Temperatura do aço antes do VOD(°C),0.0118,0.021,0.557,0.577,-0.030,0.053,0.502954
Peso de aço antes do VOD(kg),0.0008,0.002,0.460,0.646,-0.003,0.004,NaN
Borda livre da panela(mm),0.0051,0.003,1.774,0.076,-0.001,0.011,0.501279
Vida de plug da panela,-0.7167,0.360,-1.990,0.047,-1.423,-0.011,0.328118


In [ ]:
## variáveis novas afetam muito pouco o resultado.

In [ ]:
##Pressão média de vácuo durante o sopro de O2(mbar)
# Pressão final de vácuo na descarburação(mbar) E Pressão final de vácuo do sopro de O2(mbar): altera??
##

In [173]:
std_ = dt[['Temperatura máxima dos gases durante o sopro de O2(°C)','Tempo total de descarburação(min)',
'Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)']].std()
mean_ = dt[['Temperatura máxima dos gases durante o sopro de O2(°C)','Tempo total de descarburação(min)',
'Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)']].mean()
cv = std_/mean_
cv

Temperatura máxima dos gases durante o sopro de O2(°C)    0.174721
Tempo total de descarburação(min)                         0.193713
Pressão final de vácuo do sopro de O2(mbar)               0.500615
Tempo de depressão de vácuo(min)                          0.799194
Pressão final de vácuo na descarburação(mbar)             3.983188
dtype: float64

In [ ]:
## variáveis novas de beta_2 variam muito a ponto de o modelo não identificar qualquer relação siginifcante

In [154]:
tt2[columns_control]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3),Temperatura máxima dos gases durante o sopro de O2(°C),Pressão final de vácuo do sopro de O2(mbar),Tempo de depressão de vácuo(min),Pressão final de vácuo na descarburação(mbar)
0,6.825189,693.316496,23.0,5.087555,176.462269,278.64,248.17,5.233,2.0


In [155]:
stp

[[4.9, 465.6, 16.717, 5.97, 184.0, 278.64, 248.17, 5.233, 2.0]]

In [36]:
#L2 regularization:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import warnings

warnings.filterwarnings('ignore')

# Custom Huber loss function for logistic regression with L2 regularization
def huber_loss(params, X, y, delta=1.0, lambda_=0.01):
    """Calculate Huber loss for logistic regression with L2 regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    
    # L2 regularization term
    regularization_term = lambda_ * np.sum(params**2)
    
    return (np.sum(loss) / n) + regularization_term

def huber_loss_grad(params, X, y, delta=1.0, lambda_=0.01):
    """Gradient of Huber loss for logistic regression with L2 regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    
    # Gradient of L2 regularization
    reg_grad = 2 * lambda_ * params
    
    return grad + reg_grad

# Required columns for the model
required_columns = [
    'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
    'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
    'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
    'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
    'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)', 'Temperatura máxima dos gases durante o sopro de O2(°C)',
    'Tempo total de descarburação(min)', 'Pressão final de vácuo do sopro de O2(mbar)',
    'Tempo de depressão de vácuo(min)', 'Pressão final de vácuo na descarburação(mbar)'
]

# Define control DataFrame
columns_control = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)', 'Temperatura máxima dos gases durante o sopro de O2(°C)',
    'Pressão final de vácuo do sopro de O2(mbar)', 'Tempo de depressão de vácuo(min)',
    'Pressão final de vácuo na descarburação(mbar)',
]

# Example control values
stp = [[ 6.3,	457.0,	18.4,	5.99,	213.0,	268.63,	196.14,	3.967,	1.93]]
contr = pd.DataFrame(stp, columns=columns_control)

# Prepare input DataFrame
ent = teste[['Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
             'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
             'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
             'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
             'Tempo total de descarburação(min)']]

# Combine control and input data
contr.reset_index(drop=True, inplace=True)
tt2 = pd.concat([ent, contr], axis=1)
tt2.fillna(0, inplace=True)
tt2 = pd.DataFrame(tt2.sum()).T

# Prepare test data for predictions
tt2_test = tt2[columns_control]

# Prepare Data for training
x_ = dt[required_columns]
y_ = dt['opt']

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.1, random_state=42)

# Set the regularization strength
lambda_ = 0.01

# Optimize using Huber loss with L2 regularization
initial_params = np.zeros(x_train.shape[1])
result = minimize(fun=huber_loss, x0=initial_params, args=(x_train.values, y_train.values, 1.0, lambda_),
                  jac=huber_loss_grad, method='BFGS')

optimal_params = result.x

# Predicting on tt2
tt2_features = tt2.values
y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

# Display predictions
tt2['Predicted_Probability'] = y_pred_tt2_prob
tt2['Predicted_Class'] = y_pred_tt2_binary

print("Predictions on tt2:")
print(tt2[['Predicted_Probability', 'Predicted_Class']])


Predictions on tt2:
   Predicted_Probability  Predicted_Class
0           3.944923e-45                0


In [37]:
teste.iloc[ent_.index][columns_control].values

array([[  6.3  , 457.   ,  18.4  ,   5.99 , 213.   , 268.63 , 196.14 ,
          3.967,   1.93 ]])

In [38]:
teste.iloc[ent_.index]['opt']

14    0
Name: opt, dtype: int64

In [633]:
# With Hubber Loss new:
#Não OK
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils import resample  # Added import for resample
from scipy.optimize import minimize
import time
import warnings
import gc

# Custom Huber loss function for logistic regression
def huber_loss(params, X, y, delta=1.0):
    """Calculate Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    """Gradient of Huber loss for logistic regression."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Pressão final de vácuo do sopro de O2(mbar)', 'Tempo de depressão de vácuo(min)',
            'Pressão final de vácuo na descarburação(mbar)',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)', 'Temperatura máxima dos gases durante o sopro de O2(°C)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        # Split the data
        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.1)

        # Optimize using Huber loss
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Fit the Logit model with robust standard errors
        logit_model = sm.Logit(y_train_3, x_train_3)
        results = logit_model.fit(method='bfgs', cov_type='HC3')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Bootstrap to get p-values
        def bootstrap_p_values(x, y, n_iterations=1000):
            p_values = []
            for _ in range(n_iterations):
                x_resampled, y_resampled = resample(x, y)
                model = sm.Logit(y_resampled, x_resampled).fit(disp=0)
                p_values.append(model.pvalues)
            return np.mean(p_values, axis=0)

        bootstrapped_p_values = bootstrap_p_values(x_train_3, y_train_3)

        # Add bootstrapped p-values to the results DataFrame
        df_results['Bootstrap_P>|z|'] = bootstrapped_p_values

        # Filter significant variables
        impact_var2 = df_results[df_results['Bootstrap_P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_tt2_prob >= 0.99) and np.all(y_pred_tt2_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          2.655587e-150                0
         Current function value: 0.479161
         Iterations: 35
         Function evaluations: 51
         Gradient evaluations: 42
Iteration: 2, Time elapsed: 4.69 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.704228                1
         Current function value: 0.489041
         Iterations: 35
         Function evaluations: 49
         Gradient evaluations: 40
Iteration: 3, Time elapsed: 9.38 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0           8.834555e-77                0
         Current function value: 0.482103
         Iterations: 35
         Function evaluations: 50
         Gradient evaluations: 41
Iteration: 4, Time elapsed: 14.56 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          5.833187e-138                0
         Current 

In [27]:
## WITH HUBBER_LOSS:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time
import warnings
import gc

# Custom Huber loss function for logistic regression
def huber_loss(params, X, y, delta=1.0):
    """Calculate Huber loss for logistic regression."""
    # Convert inputs to numpy arrays for safety
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    """Gradient of Huber loss for logistic regression."""
    # Convert inputs to numpy arrays for safety
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {'Tempo abaixo de 10 mbar(min)': (9.650573007040464, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.5215818863059383, 6.8251891508760565),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.3164964745301),
    'Volume de O2 soprado(m3)': (115.39118253104017, 248.45226169596572),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T
        
        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela','Pressão final de vácuo do sopro de O2(mbar)','Tempo de depressão de vácuo(min)','Pressão final de vácuo na descarburação(mbar)',
            'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)','Temperatura máxima dos gases durante o sopro de O2(°C)'
        ]

        tt2_test = tt2[required_columns]

        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Optimize using Huber loss
        initial_params = np.zeros(x_train_3.shape[1])
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_, x_)
        
        results = logit_model.fit(method='bfgs', cov_type='HC3')#, C =0.01)

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition
        if np.all(y_pred_tt2_prob >= 0.99) and np.all(y_pred_tt2_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          1.454523e-232                0
         Current function value: 0.485102
         Iterations: 35
         Function evaluations: 51
         Gradient evaluations: 42
Iteration: 2, Time elapsed: 0.15 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          4.134748e-129                0
         Current function value: 0.485102
         Iterations: 35
         Function evaluations: 51
         Gradient evaluations: 42
Iteration: 3, Time elapsed: 0.28 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0                0.02418                0
         Current function value: 0.485102
         Iterations: 35
         Function evaluations: 51
         Gradient evaluations: 42
Iteration: 4, Time elapsed: 0.48 seconds
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0          5.313035e-231                0
         Current f

In [53]:
# custom regularization:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time
import warnings
import gc

# Custom regularization function to discourage low coefficients
def custom_regularization(params, threshold=0.1, lambda_=0.01):
    """Custom regularization function to discourage low coefficients."""
    return lambda_ * np.sum(np.clip(np.abs(params) - threshold, 0, None) ** 2)

# Custom Huber loss function for logistic regression with regularization
def huber_loss(params, X, y, delta=1.0, lambda_=0.01, threshold=0.1):
    """Calculate Huber loss for logistic regression with custom regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Huber loss calculation
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    
    # Custom regularization term
    regularization_term = custom_regularization(params, threshold, lambda_)
    
    return (np.sum(loss) / n) + regularization_term

# Gradient of Huber loss with regularization
def huber_loss_grad(params, X, y, delta=1.0, lambda_=0.01, threshold=0.1):
    """Gradient of Huber loss for logistic regression with custom regularization."""
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))  # Logistic function
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    # Gradient calculation
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    
    # Gradient of custom regularization
    reg_grad = lambda_ * np.where(np.abs(params) < threshold, 0, params - threshold * np.sign(params))
    
    grad += reg_grad
    
    return grad

# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 30
iteration = 0

# Define constraints
constraints = {
    'Tempo abaixo de 10 mbar(min)': (9.65, 23),
    'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.52, 6.82),
    'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.32),
    'Volume de O2 soprado(m3)': (115.39, 248.45),
    'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
}

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break

        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for control DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T

        # Apply constraints efficiently
        for col, (a_min, a_max) in constraints.items():
            if col in tt2.columns:
                tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

        # Required columns for model
        required_columns = [
            'Teor de C antes do VOD(%)', 'Teor de Cr antes do VOD(%)',
            'Teor de Si antes do VOD(%)', 'Teor de Mn antes do VOD(%)',
            'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
            'Temperatura do aço antes do VOD(°C)', 'Peso de aço antes do VOD(kg) ',
            'Borda livre da panela(mm)', 'Vida de plug da panela', 'Vida da panela',
            'Pressão final de vácuo do sopro de O2(mbar)', 'Tempo de depressão de vácuo(min)',
            'Pressão final de vácuo na descarburação(mbar)', 'Vazão média de argônio durante o sopro de O2(Nm3/h)',
            'Pressão média de vácuo durante o sopro de O2(mbar)',
            'Tempo abaixo de 10 mbar(min)',
            'Vazão média de argônio durante a descarburação(Nm3/h)',
            'Volume de O2 soprado(m3)', 'Temperatura máxima dos gases durante o sopro de O2(°C)'
        ]

        tt2_test = tt2[required_columns]
        print("Shape of tt2_test:", tt2_test.shape)

        # Prepare training data
        x_ = dt[required_columns]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

        # Optimize using Huber loss with custom regularization
        initial_params = np.zeros(x_train_3.shape[1])
        lambda_ = 0.01  # Regularization strength
        threshold = 0.1  # Threshold for discouraging low coefficients
        result = minimize(fun=huber_loss, x0=initial_params, args=(x_train_3.values, y_train_3.values, 1.0, lambda_, threshold),
                          jac=huber_loss_grad, method='BFGS')

        optimal_params = result.x
        print("Optimal parameters:", optimal_params)

        # Predicting on tt2
        tt2_features = tt2_test[required_columns].values
        print("Shape of tt2_features:", tt2_features.shape)

        y_pred_tt2_prob = 1 / (1 + np.exp(-np.dot(tt2_features, optimal_params)))
        y_pred_tt2_binary = (y_pred_tt2_prob >= 0.5).astype(int)

        # Display predictions for tt2
        tt2['Predicted_Probability'] = y_pred_tt2_prob
        tt2['Predicted_Class'] = y_pred_tt2_binary

        print("Predictions on tt2:")
        print(tt2[['Predicted_Probability', 'Predicted_Class']])

        # Fit the Logit model from Statsmodels
        logit_model = sm.Logit(y_train, x_train)
        results = logit_model.fit(method='bfgs', cov_type='HC3')

        # Extract coefficients and compute exponentiated coefficients
        co = results.params
        exp_coeff = np.exp(co) / (np.exp(co) + 1)
        exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

        # Extract results summary
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff_df

        # Filter significant variables
        impact_var2 = df_results[df_results['P>|z|'] < 0.05]

        # Calculate results
        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']
            if i in constraints:
                correction = tt2_test[i].median()
                result = (value - 0.5) * correction if value != 0.5 else 0
                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list).T
        results_df_.columns = results_df_.iloc[0]
        results_df_ = results_df_[1:]

        # Common columns
        common_columns = list(set(constraints.keys()) & set(results_df_.columns))
        contr_common = contr[common_columns]
        results_common = results_df_[common_columns]

        # Combine and update
        combined_df = pd.concat([contr_common, results_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        final_df = final.T
        final_df.columns = final_df.iloc[0]
        final_df = final_df[1:]

        contr.update(final_df[common_columns])
        contr.reset_index(drop=True, inplace=True)

        # Garbage collection
        gc.collect()

        # Check condition for exiting loop
        if np.all(y_pred_tt2_prob >= 0.99) and np.all(y_pred_tt2_binary == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Iteration: 1, Time elapsed: 0.00 seconds
Shape of tt2_test: (1, 20)
Optimal parameters: [ 0.09089081  0.13478997  0.00485992  0.03019381 -0.16912516  0.06671971
 -0.00780909 -0.00030494  0.00789611  0.24138579 -0.09231389 -0.00826542
 -0.26070116  0.20729411  0.12825195  0.02298859 -0.04528288 -0.18070481
  0.02476444  0.00368772]
Shape of tt2_features: (1, 20)
Predictions on tt2:
   Predicted_Probability  Predicted_Class
0               0.999041                1
         Current function value: 0.492254
         Iterations: 35
         Function evaluations: 50
         Gradient evaluations: 41


In [54]:
print(y_pred_tt2_prob, ent_.index)

[0.99904065] Index([14], dtype='int64')


In [55]:
stp_n = tt2[columns_control].values
stp_n

array([[  3.52      , 693.32      ,  23.        ,   7.87      ,
        241.41352144, 268.63      , 196.14      ,   3.967     ,
          1.93      ]])

In [56]:
tt2[columns_control]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3),Temperatura máxima dos gases durante o sopro de O2(°C),Pressão final de vácuo do sopro de O2(mbar),Tempo de depressão de vácuo(min),Pressão final de vácuo na descarburação(mbar)
0,3.52,693.32,23.0,7.87,241.413521,268.63,196.14,3.967,1.93


In [57]:
stp

[[6.3, 457.0, 18.4, 5.99, 213.0, 268.63, 196.14, 3.967, 1.93]]

In [34]:
teste.iloc[ent_.index][columns_control]

,Vazão média de argônio durante o sopro de O2(Nm3/h),Pressão média de vácuo durante o sopro de O2(mbar),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Volume de O2 soprado(m3),Temperatura máxima dos gases durante o sopro de O2(°C),Pressão final de vácuo do sopro de O2(mbar),Tempo de depressão de vácuo(min),Pressão final de vácuo na descarburação(mbar)
14,6.3,457.0,18.4,5.99,213.0,268.63,196.14,3.967,1.93


In [33]:
teste.iloc[ent_.index]['opt']

14    0
Name: opt, dtype: int64

In [273]:
teste.iloc[ent_.index]

,CORRIDA,Teor de C antes do VOD(%),Teor de Cr antes do VOD(%),Teor de Si antes do VOD(%),Teor de Mn antes do VOD(%),Teor de Ni antes do VOD(%),Teor de Mo antes do VOD(%),Temperatura do aço antes do VOD(°C),Peso de aço antes do VOD(kg),Borda livre da panela(mm),...,Tempo total de descarburação(min),Tempo abaixo de 10 mbar(min),Vazão média de argônio durante a descarburação(Nm3/h),Pressão final de vácuo na descarburação(mbar),%CO no final da descarburação,Volume de O2 soprado(m3),Teor de C após a descarburação(%),Temperatura do aço após a descarburação(°C),cluster,opt
13,820633,0.49,12.5,0.12,0.2,5.9,2.01,1552,27700.0,1002,...,24.8,18.95,7.57,1.99,0.0,173.0,0.006,1671.0,0,0


In [43]:
results_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    opt   No. Observations:                  459
Model:                          Logit   Df Residuals:                      438
Method:                           MLE   Df Model:                           20
Date:                Fri, 11 Oct 2024   Pseudo R-squ.:                  0.1061
Time:                        07:37:53   Log-Likelihood:                -225.94
converged:                      False   LL-Null:                       -252.76
Covariance Type:                  HC3   LLR p-value:                 6.555e-05
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Teor de C antes do VOD(%)                                  0.1223      1.345      0.091      0.928      -2.513       2.758
Teor de Cr antes do VOD(%)                                 0.1587      0.408      0.389      0.697      -0.641       0.958
Teor de Si antes do VOD(%)                                 0.4017      3.399      0.118      0.906      -6.259       7.063
Teor de Mn antes do VOD(%)                                 0.6640      2.438      0.272      0.785      -4.115       5.443
Teor de Ni antes do VOD(%)                                -1.7140      0.901     -1.902      0.057      -3.480       0.052
Teor de Mo antes do VOD(%)                                 0.2329      1.417      0.164      0.869      -2.544       3.010
Temperatura do aço antes do VOD(°C)                        0.0016      0.004      0.376      0.707      -0.007       0.010
Peso de aço antes do VOD(kg)                            2.564e-05      0.000      0.161      0.872      -0.000       0.000
Borda livre da panela(mm)                                  0.0017      0.001      2.434      0.015       0.000       0.003
Vida de plug da panela                                     0.0283      0.065      0.436      0.663      -0.099       0.156
Vida da panela                                             0.0460      0.044      1.048      0.295      -0.040       0.132
Vazão média de argônio durante o sopro de O2(Nm3/h)        0.0062      0.176      0.035      0.972      -0.339       0.351
Pressão média de vácuo durante o sopro de O2(mbar)         0.0087      0.003      2.878      0.004       0.003       0.015
Tempo abaixo de 10 mbar(min)                               0.0260      0.044      0.595      0.552      -0.060       0.112
Vazão média de argônio durante a descarburação(Nm3/h)      0.0311      0.174      0.179      0.858      -0.310       0.372
Volume de O2 soprado(m3)                                  -0.0032      0.009     -0.340      0.734      -0.021       0.015
Temperatura máxima dos gases durante o sopro de O2(°C)     0.0020      0.004      0.569      0.569      -0.005       0.009
Tempo total de descarburação(min)                         -0.0381      0.041     -0.938      0.348      -0.118       0.042
Pressão final de vácuo do sopro de O2(mbar)               -0.0085      0.003     -3.123      0.002      -0.014      -0.003
Tempo de depressão de vácuo(min)                          -0.0776      0.041     -1.883      0.060      -0.158       0.003
Pressão final de vácuo na descarburação(mbar)              0.0166      0.008      2.147      0.032       0.001       0.032
==========================================================================================================================
"""

In [391]:
results_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    opt   No. Observations:                  511
Model:                          Logit   Df Residuals:                      491
Method:                           MLE   Df Model:                           19
Date:                Fri, 04 Oct 2024   Pseudo R-squ.:                  0.1100
Time:                        10:56:01   Log-Likelihood:                -247.89
converged:                      False   LL-Null:                       -278.52
Covariance Type:                  HC3   LLR p-value:                 2.434e-06
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Teor de C antes do VOD(%)                                  0.2258      1.331      0.170      0.865      -2.382       2.834
Teor de Cr antes do VOD(%)                                 0.2067      0.395      0.523      0.601      -0.568       0.982
Teor de Si antes do VOD(%)                                 0.2937      3.296      0.089      0.929      -6.166       6.754
Teor de Mn antes do VOD(%)                                 0.4469      2.414      0.185      0.853      -4.285       5.179
Teor de Ni antes do VOD(%)                                -1.8407      0.826     -2.229      0.026      -3.459      -0.222
Teor de Mo antes do VOD(%)                                 0.1383      1.483      0.093      0.926      -2.769       3.045
Temperatura do aço antes do VOD(°C)                        0.0010      0.004      0.259      0.796      -0.006       0.008
Peso de aço antes do VOD(kg)                            4.287e-05      0.000      0.284      0.777      -0.000       0.000
Borda livre da panela(mm)                                  0.0019      0.001      2.736      0.006       0.001       0.003
Vida de plug da panela                                     0.0236      0.063      0.373      0.709      -0.101       0.148
Vida da panela                                             0.0258      0.042      0.611      0.541      -0.057       0.108
Pressão final de vácuo do sopro de O2(mbar)               -0.0086      0.003     -3.335      0.001      -0.014      -0.004
Tempo de depressão de vácuo(min)                          -0.0958      0.045     -2.131      0.033      -0.184      -0.008
Pressão final de vácuo na descarburação(mbar)              0.0227      0.009      2.652      0.008       0.006       0.039
Vazão média de argônio durante o sopro de O2(Nm3/h)        0.0157      0.170      0.092      0.927      -0.318       0.349
Pressão média de vácuo durante o sopro de O2(mbar)         0.0094      0.003      3.110      0.002       0.003       0.015
Tempo abaixo de 10 mbar(min)                               0.0131      0.044      0.299      0.765      -0.073       0.099
Vazão média de argônio durante a descarburação(Nm3/h)      0.0494      0.165      0.300      0.764      -0.273       0.372
Volume de O2 soprado(m3)                                  -0.0054      0.008     -0.677      0.498      -0.021       0.010
Temperatura máxima dos gases durante o sopro de O2(°C)     0.0024      0.003      0.699      0.484      -0.004       0.009
==========================================================================================================================
"""

In [156]:
dt.shape

(511, 30)

In [ ]:
#Quando adicionamos as outras variáveis, o resultado passa a depender exclusivamente do processo, sendo independente dos teores químicos da matéria prima.
# Uma das três variáveis , Pressão final de vácuo,Tempo de depressão de vácuo(min) e Pressão final de vácuo na descarburação(mbar) acabam 'absorvendo' ou compensando a significância do tempo abaixo de 10(mbar).
# Pressão média de vácuo passa a ser estatisticamente significante.

In [ ]:
#modelo, expectativa, aumentar pressão - aumentar acerto

In [83]:
df_results

,coef,std err,z,P>|z|,[0.025,0.975],exp_coeff
Teor de C antes do VOD(%),0.225800,1.331,0.170,0.865,-2.382,2.834,0.556203
Teor de Cr antes do VOD(%),0.206700,0.395,0.523,0.601,-0.568,0.982,0.551492
Teor de Si antes do VOD(%),0.293700,3.296,0.089,0.929,-6.166,6.754,0.572906
Teor de Mn antes do VOD(%),0.446900,2.414,0.185,0.853,-4.285,5.179,0.609909
Teor de Ni antes do VOD(%),-1.840700,0.826,-2.229,0.026,-3.459,-0.222,0.136963
Teor de Mo antes do VOD(%),0.138300,1.483,0.093,0.926,-2.769,3.045,0.534522
Temperatura do aço antes do VOD(°C),0.001000,0.004,0.259,0.796,-0.006,0.008,0.500244
Peso de aço antes do VOD(kg),0.000043,0.000,0.284,0.777,-0.000,0.000,NaN
Borda livre da panela(mm),0.001900,0.001,2.736,0.006,0.001,0.003,0.500471
Vida de plug da panela,0.023600,0.063,0.373,0.709,-0.101,0.148,0.505899


In [ ]:
# colocar as novas variáveis no setup.
# VOD: tempo sem segundo sopro 3.8 hrs , uns 15 min por, não usar
# redução de reprocesso.
#Usar o segundo modelo, CUSTOM REGULARIZATION.
#juntar